<a href="https://colab.research.google.com/github/chrdrn/dbd25-nfty_scheduler/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ntfy Umfrage-Reminder - Live-Demo

Dieses Notebook ist eine Live-Demo des ntfy Umfrage-Reminder-Skripts, das in [diesem Repository](https://github.com/chrdrn/dbd25-nfty_scheduler) gehostet wird.
Dabei erfüllt es zwei zentrale Funktionen:
- Den Prozess der Planung und des Versands von ntfy-Benachrichtigungen nachzuvollziehen bzw. anhand vordefinierter Parameter zu testen.
- Eine Plattform, um die notwendigen Konfigurationen für eure eigenen ntfy-Benachrichtigungen vorzunehmen und zu exportieren.


## Colab-Setup
Lade die Default-Konfiguration aus dem GitHub-Repository.


In [18]:
!git clone https://github.com/chrdrn/dbd25-nfty_scheduler/
%cd dbd25-nfty_scheduler

Cloning into 'dbd25-nfty_scheduler'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 92 (delta 34), reused 77 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 43.43 KiB | 2.55 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/dbd25-nfty_scheduler/dbd25-nfty_scheduler/dbd25-nfty_scheduler/dbd25-nfty_scheduler


## Aufbau des Helpers
Der Helper besteht aus drei Teilen bzw. zentralen Funktionen: **Konfiguration**, **Planung** und **Versand**.

- `config.py` und `config/*.env`: speichern Standardwerte und die ntfy-Einstellungen.
- `run.py` ist das Hauptprogramm. Es liest die Argumente und führt die passenden Schritte aus.
  - `plan`: erstellt einen Zeitplan und speichert ihn als `out/<id>_schedule.json`.
  - `--dry-run`: zeigt nur eine Vorschau, es wird nichts gesendet.
  - `--explain`: erklärt kurz, warum die Zeitpunkte so gewählt wurden.
  - `send`: liest die `.env`, lädt den Schedule und verschickt die Benachrichtigungen.

### Argumente und Parameter (kurz)
| Argument | Typ / Beispiel | Was es bewirkt |
|---|---:|---|
| start | Datum, z.B. `2026-01-21` | Erster Tag, an dem Erinnerungen geplant werden (Beginn des Zeitraums). |
| end | Datum, z.B. `2026-01-22` | Letzter Tag, an dem Erinnerungen geplant werden (Ende des Zeitraums). |
| per_day | Ganzzahl, z.B. `5` | Wie viele Erinnerungen pro Tag gesendet werden sollen. |
| min_gap | Minuten, z.B. `60` | Minimale Pause zwischen zwei Erinnerungen am selben Tag (in Minuten). |
| mode | `interval` oder `windows` | Verteilungsmodus: `interval` = gleichmäßig im Zeitbereich; `windows` = in festgelegten Zeitfenstern. |
| interval | `HH:MM-HH:MM`, z.B. `08:00-20:00` | Nur bei `mode="interval"`: Zeitspanne, innerhalb der Erinnerungen liegen dürfen. |
| windows | Komma-getrennte Fenster, z.B. `08:00-10:00,10:00-12:00` | Nur bei `mode="windows"`: Liste von Zeitfenstern, in denen Erinnerungen platziert werden. |
| seed | Ganzzahl oder `None`, z.B. `123` | Zufalls-Seed für reproduzierbare Pläne. Gleiches Seed -> gleiche Planung; `None` -> echte Zufälligkeit. |
| participant_id | Text, z.B. `gruppeA` | Kennung der Gruppe; wird z.B. im Dateinamen und in URLs verwendet. |
| out_file | Pfad, z.B. `out/gruppeA_schedule.json` | Datei, in die der erzeugte Zeitplan geschrieben wird. |


## Beispiel: Dry-Run mit Defaults
Ein Dry-Run simuliert die Ausführung der zentralen Funktionen: Zeiten werden berechnet und als Vorschau ausgegeben (oder in eine Schedule-Datei geschrieben), aber es werden keine ntfy-Nachrichten verschickt.

Der Input bzw. die Default-Parameter stammen aus den (Template-)Konfigurationsdateien (z.B. `config.py` und `config/*.env` / `config/ntfy.env.example`) und werden ggf. durch Kommandozeilenargumente (`--start`, `--per-day`, `--interval` usw.) überschrieben.


### Dry-Run: plan
Beispiel für die Erstellung eines Zeitplans auf Basis der Default-Parameter (inklusive Konsolen-Output):

```bash
python run.py plan --dry-run --explain
```


In [19]:
import subprocess

result = subprocess.run(
    ["python", "run.py", "plan", "--dry-run", "--explain"],
    check=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)
print(result.stdout)


[explain] erlaubte Minuten pro Tag: 720 (aus 1 Fenster(n))
[explain] 2026-01-21: gewählt -> 09:33, 14:01, 17:12, 18:28, 19:40
[explain] 2026-01-22: gewählt -> 08:00, 09:22, 10:22, 13:22, 16:08
Geplante Zeitpunkte:
  ID 1: 2026-01-21 #1/5 @ 09:33 (2026-01-21T09:33)
  ID 2: 2026-01-21 #2/5 @ 14:01 (2026-01-21T14:01)
  ID 3: 2026-01-21 #3/5 @ 17:12 (2026-01-21T17:12)
  ID 4: 2026-01-21 #4/5 @ 18:28 (2026-01-21T18:28)
  ID 5: 2026-01-21 #5/5 @ 19:40 (2026-01-21T19:40)
  ID 6: 2026-01-22 #1/5 @ 08:00 (2026-01-22T08:00)
  ID 7: 2026-01-22 #2/5 @ 09:22 (2026-01-22T09:22)
  ID 8: 2026-01-22 #3/5 @ 10:22 (2026-01-22T10:22)
  ID 9: 2026-01-22 #4/5 @ 13:22 (2026-01-22T13:22)
  ID 10: 2026-01-22 #5/5 @ 16:08 (2026-01-22T16:08)

Gespeichert in: out/schedule.json

[dry-run] Kein Versand.



### Dry-Run: send
Beispiel für den Versand einer Nachricht auf Basis der Default-Parameter (inklusive Konsolen-Output):

```bash
python run.py --env-file config/ntfy.env.example send 1 --dry-run --explain
```


In [20]:
import json
from pathlib import Path
from run import _build_payload
from ntfy_reminder.send import load_env_file

schedule_path = Path("out/schedule.json")
env_path = Path("config/ntfy.env.example")

if not schedule_path.exists():
    print(f"Schedule fehlt: {schedule_path} (bitte zuerst den Plan-Dry-Run oben ausfuehren)")
elif not env_path.exists():
    print(f"Env fehlt: {env_path}")
else:
    schedule = json.loads(schedule_path.read_text())
    item = next((it for it in schedule.get("items", []) if int(it.get("id", -1)) == 1), None)
    if not item:
        print("Reminder ID 1 nicht im Schedule gefunden.")
    else:
        payload = _build_payload(schedule, item)
        env = load_env_file(str(env_path))
        survey_tpl = env.get("SURVEY_URL_TEMPLATE", "").strip()
        if survey_tpl:
            payload["url"] = survey_tpl.format(**payload)
        title_tpl = env.get("NTFY_TITLE", "")
        msg_tpl = env.get("NTFY_MESSAGE", "")
        title = title_tpl.format(**payload) if title_tpl else ""
        body = msg_tpl.format(**payload) if msg_tpl else ""

        preview = {
            "title": title,
            "message": body,
            "url": payload.get("url", ""),
        }
        print(json.dumps(preview, ensure_ascii=False, indent=2))


{
  "title": "Umfrage-Reminder 1/5",
  "message": "Bitte nimm kurz an der Umfrage teil: https://www.soscisurvey.de/dbd25-YIP2tX54JZ/?r=1",
  "url": "https://www.soscisurvey.de/dbd25-YIP2tX54JZ/?r=1"
}


## Individualisierung

In diesem Abschnitt stellt ihr eure eigenen Parameter und Einstellungen ein, um den Reminder-Planer für eure eigene Gruppe zu konfigurieren.
Bitte achtet auf folgende Punkte:
- Nutzt für die `participant_id` das Gruppenkürzel aus der [heutigen Präsentation](https://chrdrn.github.io/dbd_2025/slides/slides-03.html#/get-started).
- Das finale Thema (bzw. dessen Link) wird erst in der Mail verschickt. Benutzt bitte trotzdem einen möglichst individualisierten Link, da die Themen grundsätzlich öffentlich zugänglich sind.


In [22]:
# Individualisierte Werte (anpassen)
participant_id = "gruppe_test"  # Name deiner Gruppe (wird im Dateinamen verwendet)
start = "2026-01-21"  # Startdatum (YYYY-MM-DD)
end = "2026-01-22"  # Enddatum (YYYY-MM-DD)
per_day = 5  # Erinnerungen pro Tag
min_gap = 60  # Mindestabstand in Minuten
seed = 123  # Gleicher Seed = gleicher Plan; None für Zufall
mode = "interval"  # Verteilungsmodus: interval oder windows
interval = "08:00-20:00"  # Zeitspanne für interval
windows = "08:00-10:00,10:00-12:00,12:00-14:00,14:00-16:00,16:00-18:00"  # Zeitfenster für windows
out_file = f"out/{participant_id}_schedule.json"  # Ausgabe-Datei


### Erzeugung von `plan` mit eigenen Parametern
Erzeugt euren Schedule mit den oben gesetzten Werten und zeigt den Konsolen-Output.


#### Vorschau des Inputs (Konsolen-Befehl)

Der Chunk generiert auf Basis der oben angegebenen Parameter den Konsolen-Befehl, der ausgeführt werden soll.


In [23]:
import subprocess

cmd = [
    "python",
    "run.py",
    "--start",
    start,
    "--end",
    end,
    "--per-day",
    str(per_day),
    "--min-gap",
    str(min_gap),
    "--mode",
    mode,
    "--participant-id",
    participant_id,
    "--out",
    out_file,
]

if seed is not None:
    cmd += ["--seed", str(seed)]

if mode == "windows":
    cmd += ["--windows", windows]
else:
    cmd += ["--interval", interval]

cmd += [
    "plan",
    "--dry-run",
    "--explain",
]

subprocess.run(cmd, check=True)


CompletedProcess(args=['python', 'run.py', '--start', '2026-01-21', '--end', '2026-01-22', '--per-day', '5', '--min-gap', '60', '--mode', 'interval', '--participant-id', 'gruppe_test', '--out', 'out/gruppe_test_schedule.json', '--seed', '123', '--interval', '08:00-20:00', 'plan', '--dry-run', '--explain'], returncode=0)

#### Vorschau Output

Der Chunk zeigt den erwarteten Konsolen-Output, der bei der Ausführung des Befehls entstehen würde.


In [24]:
import copy
import subprocess

preview_out_file = f"out/{participant_id}_schedule_preview.json"

plan_cmd = copy.deepcopy(cmd)
if "--out" in plan_cmd:
    out_idx = plan_cmd.index("--out")
    plan_cmd[out_idx + 1] = preview_out_file

result = subprocess.run(
    plan_cmd,
    check=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)
print(result.stdout)


[explain] participant_id='gruppe_test' -> pid_hash_int=112847462236711
[explain] final seed=112847576257558 (base_seed=123)
[explain] Seed gesetzt auf: 112847576257558
[explain] erlaubte Minuten pro Tag: 720 (aus 1 Fenster(n))
[explain] 2026-01-21: gewählt -> 10:04, 13:13, 14:36, 16:22, 19:16
[explain] 2026-01-22: gewählt -> 08:00, 09:39, 12:21, 14:35, 16:11
Geplante Zeitpunkte:
  ID 1: 2026-01-21 #1/5 @ 10:04 (2026-01-21T10:04)
  ID 2: 2026-01-21 #2/5 @ 13:13 (2026-01-21T13:13)
  ID 3: 2026-01-21 #3/5 @ 14:36 (2026-01-21T14:36)
  ID 4: 2026-01-21 #4/5 @ 16:22 (2026-01-21T16:22)
  ID 5: 2026-01-21 #5/5 @ 19:16 (2026-01-21T19:16)
  ID 6: 2026-01-22 #1/5 @ 08:00 (2026-01-22T08:00)
  ID 7: 2026-01-22 #2/5 @ 09:39 (2026-01-22T09:39)
  ID 8: 2026-01-22 #3/5 @ 12:21 (2026-01-22T12:21)
  ID 9: 2026-01-22 #4/5 @ 14:35 (2026-01-22T14:35)
  ID 10: 2026-01-22 #5/5 @ 16:11 (2026-01-22T16:11)

Gespeichert in: out/gruppe_test_schedule_preview.json

[dry-run] Kein Versand.



#### Erstellung schedule.json
Zeigt den Inhalt der erzeugten Datei als JSON, damit ihr seht, was gespeichert wurde.


In [25]:
import json
from pathlib import Path

schedule_path = Path(out_file)
if schedule_path.exists():
    data = json.loads(schedule_path.read_text())
    print(json.dumps(data, ensure_ascii=False, indent=2))
else:
    print(f"Datei nicht gefunden: {schedule_path}")


{
  "generated_at": "2026-01-22T22:22:03",
  "start_date": "2026-01-21",
  "end_date": "2026-01-22",
  "per_day": 5,
  "min_gap_minutes": 60,
  "mode": "interval",
  "items": [
    {
      "id": 1,
      "day": "2026-01-21",
      "k": 1,
      "per_day": 5,
      "when": "2026-01-21T10:04",
      "time": "10:04"
    },
    {
      "id": 2,
      "day": "2026-01-21",
      "k": 2,
      "per_day": 5,
      "when": "2026-01-21T13:13",
      "time": "13:13"
    },
    {
      "id": 3,
      "day": "2026-01-21",
      "k": 3,
      "per_day": 5,
      "when": "2026-01-21T14:36",
      "time": "14:36"
    },
    {
      "id": 4,
      "day": "2026-01-21",
      "k": 4,
      "per_day": 5,
      "when": "2026-01-21T16:22",
      "time": "16:22"
    },
    {
      "id": 5,
      "day": "2026-01-21",
      "k": 5,
      "per_day": 5,
      "when": "2026-01-21T19:16",
      "time": "19:16"
    },
    {
      "id": 6,
      "day": "2026-01-22",
      "k": 1,
      "per_day": 5,
      "when": "2

### ntfy konfigurieren (.env)
Hier definierst du die Werte für deine Benachrichtigung und schreibst sie in `config/ntfy.env`.


#### Definition relevanter .env-Werte
Passe diese Werte an deine Gruppe an. Sie werden im nächsten Chunk in die .env geschrieben.


In [26]:
ntfy_topic = "dbd25-5hhxJAKjRM-group_test"  # Dein ntfy-Topic
ntfy_title = "Umfrage-Reminder {k}/{per_day}"  # Titelvorlage
ntfy_message = "Bitte nimm kurz an der Umfrage teil: {url}"  # Nachrichtentext

survey_url_template = "https://www.soscisurvey.de/dbd25-5hhxJAKjRM-group_test//?r={id}"  # Link-Vorlage
ntfy_server = "https://ntfy.sh"  # Server (Standard: ntfy.sh)
ntfy_markdown = "1"  # Markdown aktivieren (optional)


In [27]:
from pathlib import Path

env_path = Path("config/ntfy.env")
lines = [
    f"export NTFY_TOPIC=\"{ntfy_topic}\"",
    f"export NTFY_TITLE=\"{ntfy_title}\"",
    f"export NTFY_MESSAGE=\"{ntfy_message}\"",
]

if survey_url_template:
    lines.append(f"export SURVEY_URL_TEMPLATE=\"{survey_url_template}\"")
if ntfy_server:
    lines.append(f"export NTFY_SERVER=\"{ntfy_server}\"")
if ntfy_markdown:
    lines.append(f"export NTFY_MARKDOWN=\"{ntfy_markdown}\"")

env_text = "\n".join(lines) + "\n"
env_path.write_text(env_text)
print(f"Wrote {env_path}")


Wrote config/ntfy.env


### Optional: Testbenachrichtigung senden
Nur ausführen, wenn `config/ntfy.env` echte Werte enthält.


In [28]:
!python run.py send 1 --explain


[explain] POST https://ntfy.sh/dbd25-5hhxJAKjRM-group_test
[explain] Title: Umfrage-Reminder 1/5
[explain] Click: https://www.soscisurvey.de/dbd25-5hhxJAKjRM-group_test//?r=1
[explain] Body:
Bitte nimm kurz an der Umfrage teil: https://www.soscisurvey.de/dbd25-5hhxJAKjRM-group_test//?r=1
OK: Reminder ID 1 gesendet.


### Export (Pflicht)
Diese Dateien braucht ihr später für den Versand und zur Abgabe. Bitte exportiert beides am Ende.

1. `config/ntfy.env` (eure individualisierte .env)
2. `out/<id>_schedule.json` (der erzeugte Schedule; in diesem Notebook ist das `out_file`)

Der Download-Helper funktioniert nur in Colab.


In [29]:
from pathlib import Path

files_to_export = [Path(out_file), Path("config/ntfy.env")]

try:
    from google.colab import files
    print("Exportiere:")
    for p in files_to_export:
        if p.exists():
            print(f"- {p}")
            files.download(str(p))
        else:
            print(f"Missing: {p}")
except Exception:
    print("Not running in Colab.")


Exportiere:
- out/gruppe_test_schedule.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- config/ntfy.env


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>